In [77]:
import requests
import os
import json
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import datetime as dt
from dotenv import load_dotenv
from google.cloud import storage

# Setting up the GCP infrastructure

In [73]:
# Load environment variables from .env file
load_dotenv()

# Check if the environment variable is set
if "GOOGLE_APPLICATION_CREDENTIALS" not in os.environ:
    raise ValueError("GOOGLE_APPLICATION_CREDENTIALS not found in environment variables.")

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] 

In [74]:
raw_data_bucket = os.environ["GOOGLE_RAW_DATA_BUCKET"] 
clean_data_bucket = os.environ["GOOGLE_CLEAN_DATA_BUCKET"] 
#raw_data_bucket = "farm-screener-raw"

In [29]:
datestamp = dt.datetime.now().date()
datestamp

datetime.date(2024, 3, 9)

In [4]:
# Create a client
client = storage.Client()

In [71]:

# List buckets
print("Buckets:")
for bucket in client.list_buckets():
    print(bucket.name)

Buckets:
farm-screener-clean
farm-screener-raw


In [6]:
def upload_to_gcs(data, bucket_name, file_name):
    try:
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(file_name)
        blob.upload_from_string(json.dumps(data))
        print(f"Data uploaded to GCS: gs://{bucket_name}/{file_name}")
    except Exception as e:
        print(f"Error uploading data to GCS: {e}")

In [7]:
import requests

def download_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to download data. Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error downloading data: {e}")
        return None


In [50]:
size = 100
offset = size

while True:
    #url = "https://www.hofsuche.schweizerbauern.ch/api/de/farms?offset=24&project=vomhof&size=24"
    url = f"https://www.hofsuche.schweizerbauern.ch/api/de/farms?offset={offset}&project=vomhof&size={size}"
    data = download_data(url)
    print(f"Offset: {offset}")
    if data:
        print("Data downloaded successfully!")
        if len(data["hits"] == 0):
             break
        else:
            for item in data["hits"]:
                    # Add additional fields here as needed
                    item['timestamp'] = str(datestamp)
                    
                    # Upload the modified data item to GCS
                    file_name = f"{item['_id']}.json"  # Using the 'id' field as the filename
                    upload_to_gcs(item, raw_data_bucket, file_name)

    else:
        print("Failed to download data. Please check the URL and try again.")
    offset = offset + size


Offset: 100
Data downloaded successfully!
Data uploaded to GCS: gs://farm-screener-raw/14482.json
Data uploaded to GCS: gs://farm-screener-raw/14419.json
Data uploaded to GCS: gs://farm-screener-raw/14349.json
Data uploaded to GCS: gs://farm-screener-raw/14441.json
Data uploaded to GCS: gs://farm-screener-raw/14432.json
Data uploaded to GCS: gs://farm-screener-raw/14425.json
Data uploaded to GCS: gs://farm-screener-raw/14408.json
Data uploaded to GCS: gs://farm-screener-raw/14423.json
Data uploaded to GCS: gs://farm-screener-raw/14410.json
Data uploaded to GCS: gs://farm-screener-raw/14400.json
Data uploaded to GCS: gs://farm-screener-raw/14409.json
Data uploaded to GCS: gs://farm-screener-raw/14392.json
Data uploaded to GCS: gs://farm-screener-raw/14404.json
Data uploaded to GCS: gs://farm-screener-raw/14402.json
Data uploaded to GCS: gs://farm-screener-raw/14401.json
Data uploaded to GCS: gs://farm-screener-raw/14394.json
Data uploaded to GCS: gs://farm-screener-raw/14397.json
Data u

KeyboardInterrupt: 

In [32]:
def get_farm_data(item):
    farm ={}
    farm["farm_id"] = item["_id"]
    farm["farm_name"] = item["_source"]["farmname"]
    farm["first_name"] = item["_source"]["first_name"]
    farm["last_name"] = item["_source"]["last_name"]
    farm["street"] = item["_source"]["street"]
    farm["zip"] = item["_source"]["zip"]
    farm["city"] = item["_source"]["city"]
    farm["canton"] = item["_source"]["canton"]["short"]
    farm["telephone"] = item["_source"]["telephone"]
    farm["mobile"] = item["_source"]["mobile"]
    farm["email"] = item["_source"]["email"]
    farm["website"] = item["_source"]["website"]
    farm["facebook_link"] = item["_source"]["facebook_link"]
    farm["timestamp"] = item["timestamp"]
    
    return farm

## Access json files in GCP bucket

In [51]:
# List JSON files in the GCS bucket
storage_client = storage.Client()
bucket = storage_client.get_bucket(raw_data_bucket)


In [57]:
farm_data = []
product_offers = list()

blobs = bucket.list_blobs()
for blob in blobs:
    if blob.name.endswith('.json'):
        uri = f"gs://{raw_data_bucket}/{blob.name}"
        json_data_string = blob.download_as_string().decode("utf-8")
        json_data = json.loads(json_data_string)

        # extract data for each farm
        farm_id = json_data["_id"]
        try:
            farm_data.append(get_farm_data(json_data))
        except:
            farm_data.append(farm_id)
        offers = item["_source"]["vomhof"]["offers"]
        for offer in offers:
            product = offer["product"]
            product_id = product["id"]
            product_name = product["name"]
            availability = offer["availability"]
            if len(availability) > 0:
                product_availability = [month["name"] for month in availability]
            else:
                product_availability = "Not specified"

            product_offers.append((farm_id, product_id, product_name, product_availability))
        

In [42]:
# farm_data = []
# product_offers = list()

# for i, item in enumerate(data["hits"]):
#     farm_id = item["_id"]
#     try:
#         farm_data.append(get_farm_data(item))
#     except:
#         farm_data.append(farm_id)
#     offers = item["_source"]["vomhof"]["offers"]
#     for offer in offers:
#         product = offer["product"]
#         product_id = product["id"]
#         product_name = product["name"]
#         availability = offer["availability"]
#         if len(availability) > 0:
#             product_availability = [month["name"] for month in availability]
#         else:
#             product_availability = "Not specified"

#         product_offers.append((farm_id, product_id, product_name, product_availability))
#         #print(product)
#         #print(product_availability)

In [58]:
columns = ["farm_id", "product_id", "product_name", "product_availability"]
product_df = pd.DataFrame.from_records(product_offers, columns=columns)
product_df

,farm_id,product_id,product_name,product_availability
0,10002,2722,Eingemachtes Gemüse,Not specified
1,10002,15,Gebäck süss,Not specified
2,10002,258,Honig,Not specified
3,10002,13,Hühnereier,"[Januar, Februar, März, April, Mai, Juni, Juli..."
4,10002,12,Käse,"[Januar, Februar, März, April, Mai, September,..."
...,...,...,...,...
32324,14875,33,Milch,Not specified
32325,14875,26,Rindfleisch,Not specified
32326,14875,179,Speiseöl,Not specified
32327,14875,16,Wurstwaren,Not specified


In [80]:
product_df["farm_id"] = product_df["farm_id"].astype("int")
product_df["product_availability"] = product_df["product_availability"].astype("str") # needed for conversion to parquet
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32329 entries, 0 to 32328
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   farm_id               32329 non-null  int64 
 1   product_id            32329 non-null  object
 2   product_name          32329 non-null  object
 3   product_availability  32329 non-null  object
dtypes: int64(1), object(3)
memory usage: 1010.4+ KB


In [81]:
farm_df = pd.DataFrame.from_records(farm_data)
farm_df

,farm_id,farm_name,first_name,last_name,street,zip,city,canton,telephone,mobile,email,website,facebook_link,timestamp
0,10002,Hof Wolfenberg,Barbara & Martin,Venzin,Wolfenbergstrasse,8426,Lufingen,ZH,044 803 12 73,079 836 03 48,martin.venzin@wolfenberg.ch,http://www.wolfenberg.ch,None,2024-03-09
1,10007,Alpkäserei Risch /under Münenberg,Ueli & Martha,Bieri-Wicki,Under Münenberg 1,6162,Rengg,LU,041 480 37 75,,marbi2@bluewin.ch,http://www.alpkaeserisch.ch,None,2024-03-09
2,10009,HO DELice du Pré-Mermoud,Chantal & Daniel,Hodel,Pré-Mermoud 1,1580,Avenches,VD,,079 360 54 24,chantal.daniel@hotmail.com,https://www.hodelice.ch,,2024-03-09
3,10011,Scheidweg,Käthi & Röbi,Messmer,Hagenbuch 5,8577,Schönholzerswilen,TG,,,rkmessmer@bluewin.ch,,None,2024-03-09
4,10013,Zebuhof,Nadia & Kari,Bürgi-Schelbert,Bergstrasse 12,6424,Lauerz,SZ,041 811 18 56,079 225 29 89,info@zebuhof.ch,http://www.zebuhof.ch,https://www.facebook.com/Zebubuur/,2024-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934,9990,Laueli / Alp Wittenlauenen,Erika & Theo,Emmenegger-Bucher,Laueli 1,6174,Sörenberg,LU,041 488 00 23,078 926 06 05,,http://alpgruss.ch,https://www.facebook.com/Alpgruss-101220415403839,2024-03-09
2935,9991,Zimbel,Cornelia & Martin,Keiser-Schneider,Zimbel,6340,Baar,ZG,,079 614 79 03,,,None,2024-03-09
2936,9995,Gantlihof,"Fidel, Elisabeth & Christian",Kenel,Sonnenbergstrasse 2,6415,Arth,SZ,041 855 41 34,079 344 91 29,,http://www.gantlihof.ch,None,2024-03-09
2937,9999,Vetsch Hattenhausen,Thomas,Vetsch,Fischbachstrasse 7,8564,Hattenhausen,TG,052 763 38 32,079 318 83 65,monikavetsch@bluewin.ch,,None,2024-03-09


In [82]:
farm_df["farm_id"]= farm_df["farm_id"].astype("int")
#farm_df["timestamp"]= date = pd.to_datetime(farm_df["timestamp"])

In [83]:
farm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2939 entries, 0 to 2938
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   farm_id        2939 non-null   int64 
 1   farm_name      2939 non-null   object
 2   first_name     2939 non-null   object
 3   last_name      2939 non-null   object
 4   street         2939 non-null   object
 5   zip            2939 non-null   object
 6   city           2939 non-null   object
 7   canton         2939 non-null   object
 8   telephone      2939 non-null   object
 9   mobile         2939 non-null   object
 10  email          2939 non-null   object
 11  website        2939 non-null   object
 12  facebook_link  995 non-null    object
 13  timestamp      2939 non-null   object
dtypes: int64(1), object(13)
memory usage: 321.6+ KB


## Saving clean data to GCP bucket in parquet

In [66]:
farms_parquet_file_path = "farms.parquet"
offers_parquet_file_path = "offers.parquet"

In [79]:
# Write the DataFrame to a Parquet file
with storage_client.bucket(clean_data_bucket).blob(farms_parquet_file_path).open("wb") as file:
    pq.write_table(pa.Table.from_pandas(farm_df), file, compression='snappy')

print(f"DataFrame saved to Parquet file: gs://{clean_data_bucket}/{farms_parquet_file_path}")



DataFrame saved to Parquet file: gs://farm-screener-clean/farms.parquet


ArrowTypeError: ("Expected bytes, got a 'list' object", 'Conversion failed for column product_availability with type object')

In [84]:

with storage_client.bucket(clean_data_bucket).blob(offers_parquet_file_path).open("wb") as file:
    pq.write_table(pa.Table.from_pandas(product_df), file, compression='snappy')

print(f"DataFrame saved to Parquet file: gs://{clean_data_bucket}/{offers_parquet_file_path}")

DataFrame saved to Parquet file: gs://farm-screener-clean/offers.parquet


## EDA on collected data

In [85]:
# Read the Parquet file from GCS
with storage_client.bucket(clean_data_bucket).blob(farms_parquet_file_path).open("rb") as file:
    farms_parquet_table = pq.read_table(file)

# Convert the Parquet table to a pandas DataFrame
new_farms_df = farms_parquet_table.to_pandas()

In [86]:
new_farms_df

,farm_id,farm_name,first_name,last_name,street,zip,city,canton,telephone,mobile,email,website,facebook_link,timestamp
0,10002,Hof Wolfenberg,Barbara & Martin,Venzin,Wolfenbergstrasse,8426,Lufingen,ZH,044 803 12 73,079 836 03 48,martin.venzin@wolfenberg.ch,http://www.wolfenberg.ch,None,2024-03-09
1,10007,Alpkäserei Risch /under Münenberg,Ueli & Martha,Bieri-Wicki,Under Münenberg 1,6162,Rengg,LU,041 480 37 75,,marbi2@bluewin.ch,http://www.alpkaeserisch.ch,None,2024-03-09
2,10009,HO DELice du Pré-Mermoud,Chantal & Daniel,Hodel,Pré-Mermoud 1,1580,Avenches,VD,,079 360 54 24,chantal.daniel@hotmail.com,https://www.hodelice.ch,,2024-03-09
3,10011,Scheidweg,Käthi & Röbi,Messmer,Hagenbuch 5,8577,Schönholzerswilen,TG,,,rkmessmer@bluewin.ch,,None,2024-03-09
4,10013,Zebuhof,Nadia & Kari,Bürgi-Schelbert,Bergstrasse 12,6424,Lauerz,SZ,041 811 18 56,079 225 29 89,info@zebuhof.ch,http://www.zebuhof.ch,https://www.facebook.com/Zebubuur/,2024-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934,9990,Laueli / Alp Wittenlauenen,Erika & Theo,Emmenegger-Bucher,Laueli 1,6174,Sörenberg,LU,041 488 00 23,078 926 06 05,,http://alpgruss.ch,https://www.facebook.com/Alpgruss-101220415403839,2024-03-09
2935,9991,Zimbel,Cornelia & Martin,Keiser-Schneider,Zimbel,6340,Baar,ZG,,079 614 79 03,,,None,2024-03-09
2936,9995,Gantlihof,"Fidel, Elisabeth & Christian",Kenel,Sonnenbergstrasse 2,6415,Arth,SZ,041 855 41 34,079 344 91 29,,http://www.gantlihof.ch,None,2024-03-09
2937,9999,Vetsch Hattenhausen,Thomas,Vetsch,Fischbachstrasse 7,8564,Hattenhausen,TG,052 763 38 32,079 318 83 65,monikavetsch@bluewin.ch,,None,2024-03-09


In [87]:
# Read the Parquet file from GCS
with storage_client.bucket(clean_data_bucket).blob(offers_parquet_file_path).open("rb") as file:
    offers_parquet_table = pq.read_table(file)

# Convert the Parquet table to a pandas DataFrame
new_offers_df = offers_parquet_table.to_pandas()

In [88]:
new_offers_df

,farm_id,product_id,product_name,product_availability
0,10002,2722,Eingemachtes Gemüse,Not specified
1,10002,15,Gebäck süss,Not specified
2,10002,258,Honig,Not specified
3,10002,13,Hühnereier,"['Januar', 'Februar', 'März', 'April', 'Mai', ..."
4,10002,12,Käse,"['Januar', 'Februar', 'März', 'April', 'Mai', ..."
...,...,...,...,...
32324,14875,33,Milch,Not specified
32325,14875,26,Rindfleisch,Not specified
32326,14875,179,Speiseöl,Not specified
32327,14875,16,Wurstwaren,Not specified


In [143]:
new_offers_df["product_name"].value_counts()

product_name
Eingemachtes Gemüse    2939
Gebäck süss            2939
Honig                  2939
Hühnereier             2939
Käse                   2939
Konfitüren             2939
Milch                  2939
Rindfleisch            2939
Speiseöl               2939
Wurstwaren             2939
Zopf                   2939
Name: count, dtype: int64

In [119]:
merged_df = pd.merge(new_offers_df, new_farms_df, on='farm_id', how='left')

In [120]:
merged_df

,farm_id,product_id,product_name,product_availability,farm_name,first_name,last_name,street,zip,city,canton,telephone,mobile,email,website,facebook_link,timestamp
0,10002,2722,Eingemachtes Gemüse,Not specified,Hof Wolfenberg,Barbara & Martin,Venzin,Wolfenbergstrasse,8426,Lufingen,ZH,044 803 12 73,079 836 03 48,martin.venzin@wolfenberg.ch,http://www.wolfenberg.ch,None,2024-03-09
1,10002,15,Gebäck süss,Not specified,Hof Wolfenberg,Barbara & Martin,Venzin,Wolfenbergstrasse,8426,Lufingen,ZH,044 803 12 73,079 836 03 48,martin.venzin@wolfenberg.ch,http://www.wolfenberg.ch,None,2024-03-09
2,10002,258,Honig,Not specified,Hof Wolfenberg,Barbara & Martin,Venzin,Wolfenbergstrasse,8426,Lufingen,ZH,044 803 12 73,079 836 03 48,martin.venzin@wolfenberg.ch,http://www.wolfenberg.ch,None,2024-03-09
3,10002,13,Hühnereier,"['Januar', 'Februar', 'März', 'April', 'Mai', ...",Hof Wolfenberg,Barbara & Martin,Venzin,Wolfenbergstrasse,8426,Lufingen,ZH,044 803 12 73,079 836 03 48,martin.venzin@wolfenberg.ch,http://www.wolfenberg.ch,None,2024-03-09
4,10002,12,Käse,"['Januar', 'Februar', 'März', 'April', 'Mai', ...",Hof Wolfenberg,Barbara & Martin,Venzin,Wolfenbergstrasse,8426,Lufingen,ZH,044 803 12 73,079 836 03 48,martin.venzin@wolfenberg.ch,http://www.wolfenberg.ch,None,2024-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32346,14875,179,Speiseöl,Not specified,Guldenberg,Anita & Michael,Lienhard,Guldenberg 120,8424,Embrach,ZH,044 865 23 46,,anita.lienhard@guldenberg.ch,,https://www.facebook.com/profile.php?id=100067...,value
32347,14875,16,Wurstwaren,Not specified,Guldenberg,Anita & Michael,Lienhard,Guldenberg 120,8424,Embrach,ZH,044 865 23 46,,anita.lienhard@guldenberg.ch,,https://www.facebook.com/profile.php?id=100067...,2024-03-09
32348,14875,16,Wurstwaren,Not specified,Guldenberg,Anita & Michael,Lienhard,Guldenberg 120,8424,Embrach,ZH,044 865 23 46,,anita.lienhard@guldenberg.ch,,https://www.facebook.com/profile.php?id=100067...,value
32349,14875,185,Zopf,Not specified,Guldenberg,Anita & Michael,Lienhard,Guldenberg 120,8424,Embrach,ZH,044 865 23 46,,anita.lienhard@guldenberg.ch,,https://www.facebook.com/profile.php?id=100067...,2024-03-09


In [114]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32351 entries, 0 to 32350
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   farm_id               32351 non-null  int64 
 1   product_id            32351 non-null  object
 2   product_name          32351 non-null  object
 3   product_availability  32351 non-null  object
 4   farm_name             32351 non-null  object
 5   first_name            32351 non-null  object
 6   last_name             32351 non-null  object
 7   street                32351 non-null  object
 8   zip                   32351 non-null  object
 9   city                  32351 non-null  object
 10  canton                32351 non-null  object
 11  telephone             32351 non-null  object
 12  mobile                32351 non-null  object
 13  email                 32351 non-null  object
 14  website               32351 non-null  object
 15  facebook_link         10967 non-null

In [130]:
canton = "TG"
product = "Milch" #"Honig" # "Speiseöl"

In [131]:
farm_ids = new_offers_df.loc[new_offers_df['product_name'] == product, 'farm_id']
len(farm_ids)

2939

In [116]:
merged_df[(merged_df["canton"]==canton) & (merged_df["product_name"]== product)]
merged_df[(merged_df["canton"]==canton) & (merged_df["product_name"]== product)]

,farm_id,product_id,product_name,product_availability,farm_name,first_name,last_name,street,zip,city,canton,telephone,mobile,email,website,facebook_link,timestamp
41,10011,179,Speiseöl,Not specified,Scheidweg,Käthi & Röbi,Messmer,Hagenbuch 5,8577,Schönholzerswilen,TG,,,rkmessmer@bluewin.ch,,None,2024-03-09
118,10034,179,Speiseöl,Not specified,Weinbau Markus Held,Markus,Held,Boltshausen 9,8561,Ottoberg,TG,071 620 01 45,,kontakt@markus-held-weinbau.ch,http://markus-held-weinbau.ch,None,2024-03-09
195,10047,179,Speiseöl,Not specified,Stegackerfarm,Florian & Heidi,Dickenmann,Weidstrasse 20,8524,Uesslingen,TG,,079 638 57 63,,http://www.stegackerfarm.ch,None,2024-03-09
217,10050,179,Speiseöl,Not specified,Breitenhof,Manuel & Roman,Strupler,Untere Weinbergstrasse 14,8570,Weinfelden,TG,,079 740 38 81,info@strupler-events.ch,https://www.weinfelder-beef.ch,https://www.facebook.com/breitenhofweinfelden,2024-03-09
635,10134,179,Speiseöl,Not specified,Löwenhaus,Heinz & Irene,Oswald,Löwenhaus 8,8586,Kümmertshausen,TG,071 411 64 02,,i_h.oswald@sunrise.ch,,None,2024-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31886,9905,179,Speiseöl,Not specified,Bruggmanns-OberHofen,Thomas & Beatrice,Bruggmann,Kettstrasse 7,9542,Münchwilen,TG,,,info@bruggmanns-oberhofen.ch,http://www.bruggmanns-oberhofen.ch,None,2024-03-09
32117,9949,179,Speiseöl,Not specified,Blumenfeld Sulgen,Andrea & Markus,Baumberger-Klaus,Kirchstrasse 9,8583,Sulgen,TG,071 642 75 85,,,http://www.blumenfeld-sulgen.ch,None,2024-03-09
32238,9980,179,Speiseöl,Not specified,Hanfacker,Carmen & Roman Theo,Kreuzer,Hauptstrasse 3,8553,Eschikofen,TG,052 763 15 74,,,,None,2024-03-09
32282,9989,179,Speiseöl,Not specified,Sunnehof Beeri,Matthias,Müller,Amriswilerstrasse 92,9314,Steinebrunn,TG,071 474 72 05,,,http://www.sunnehofbeeri.ch,None,2024-03-09


In [142]:
merged_df.loc[merged_df["farm_id"]==9980, "product_name"]

32230    Eingemachtes Gemüse
32231            Gebäck süss
32232                  Honig
32233             Hühnereier
32234                   Käse
32235             Konfitüren
32236                  Milch
32237            Rindfleisch
32238               Speiseöl
32239             Wurstwaren
32240                   Zopf
Name: product_name, dtype: object